In [2]:
import json
import pandas as pd

In [12]:
conv_path = '/data/LPJ/ICML25/GraphCoder/graphgpt_dataset/gpt_dataset_construction/rtlcoder_gpt4_v1/import_for_graphgpt/conversations.json'
graph_path = '/data/LPJ/ICML25/GraphCoder/graphgpt_dataset/gpt_dataset_construction/rtlcoder_gpt4_v1/import_for_graphgpt/graph.jsonl'

In [14]:
import re

def find_module_string(text):
    # Regex pattern to find the string starting with "module" and ending with ";"
    # It uses non-greedy matching to find the shortest match
    pattern = r"module.*?;"
    
    # Using re.DOTALL to make the dot match all characters including newline
    match = re.search(pattern, text, re.DOTALL)
    
    if match:
        return match.group()  # Returns the matched string
    else:
        return None  # No match found

# Example usage
input_text = """
some initial text
module name="example1" \n   attribu\nte="   value";
some other text
module name="example2" attribute="another value";
"""

# Find the first occurrence
result = find_module_string(input_text)
print("Found substring:", result)

# If you need to find all occurrences instead of just the first one, you can use re.findall
all_matches = re.findall(r"module.*?;", input_text, re.DOTALL)
print("All found substrings:", all_matches)

Found substring: module name="example1" 
   attribu
te="   value";
All found substrings: ['module name="example1" \n   attribu\nte="   value";', 'module name="example2" attribute="another value";']


In [15]:
graph = pd.read_json(graph_path, lines=True)
with open(conv_path, 'r') as f:
    conv = json.load(f)


In [21]:
conv[0]['conversations'][0]['value']

"Given a submodules interconnection graph: \n<graph>\n, \nnodes: [{'id': 0, 'content': 'a', 'type': 'input port'}, {'id': 1, 'content': 'b', 'type': 'input port'}, {'id': 2, 'content': 'c', 'type': 'input port'}, {'id': 3, 'content': 'd', 'type': 'input port'}, {'id': 4, 'content': 'e', 'type': 'input port'}, {'id': 5, 'content': 'f', 'type': 'input port'}, {'id': 6, 'content': 'out', 'type': 'output port'}], \nedge_attrs: [], \nconnectivity: [[5, 0, 4, 1, 3, 2], [6, 6, 6, 6, 6, 6]]\n, \n\nYou are tasked with designing a Verilog module that takes in six inputs and produces three outputs. The inputs are `i_0r0`, `i_0r1`, `i_0a`, `i_1r`, `i_1a`, and `reset`. The outputs are `o_0r0`, `o_0r1`, and `i_1a`. The module should be named `tkj1m1_0`.\n\nThe module should have the following behavior:\n- When `reset` is high, all outputs should be set to 0.\n- `o_0r0` should be the output of a buffer that takes in `i_0r0`.\n- `o_0r1` should be the output of a buffer that takes in `i_0r1`.\n- `i_0a`

In [19]:
conv[0]['conversations'][1]['value']

'module buffer (input in, output out);\n  assign out = in;\nendmodule\n\nmodule AO222EHD (output out, input a, input b, input c, input d, input e, input f);\n  assign out = (a & b) | (c & d) | (e & f);\nendmodule'

In [25]:
head = find_module_string(conv[0]['conversations'][1]['value'])
conv[0]['conversations'][0]['value'] + head

"Given a submodules interconnection graph: \n<graph>\n, \nnodes: [{'id': 0, 'content': 'a', 'type': 'input port'}, {'id': 1, 'content': 'b', 'type': 'input port'}, {'id': 2, 'content': 'c', 'type': 'input port'}, {'id': 3, 'content': 'd', 'type': 'input port'}, {'id': 4, 'content': 'e', 'type': 'input port'}, {'id': 5, 'content': 'f', 'type': 'input port'}, {'id': 6, 'content': 'out', 'type': 'output port'}], \nedge_attrs: [], \nconnectivity: [[5, 0, 4, 1, 3, 2], [6, 6, 6, 6, 6, 6]]\n, \n\nYou are tasked with designing a Verilog module that takes in six inputs and produces three outputs. The inputs are `i_0r0`, `i_0r1`, `i_0a`, `i_1r`, `i_1a`, and `reset`. The outputs are `o_0r0`, `o_0r1`, and `i_1a`. The module should be named `tkj1m1_0`.\n\nThe module should have the following behavior:\n- When `reset` is high, all outputs should be set to 0.\n- `o_0r0` should be the output of a buffer that takes in `i_0r0`.\n- `o_0r1` should be the output of a buffer that takes in `i_0r1`.\n- `i_0a`

In [26]:
for i in range(len(conv)):
    module_head = find_module_string(conv[i]['conversations'][1]['value'])
    conv[i]['conversations'][0]['value'] = conv[i]['conversations'][0]['value'] + module_head

In [28]:
conv[100]['conversations'][0]['value']

"Given a submodules interconnection graph: \n<graph>\n, \nnodes: [{'id': 0, 'content': 'a', 'type': 'input port'}, {'id': 1, 'content': 'b', 'type': 'input port'}, {'id': 2, 'content': 'sum', 'type': 'output port'}, {'id': 3, 'content': 'adder_low', 'type': 'submodule'}, {'id': 4, 'content': 'adder_high', 'type': 'submodule'}], \nedge_attrs: [], \nconnectivity: [[1, 0, 1, 0, 4, 0, 1, 3], [3, 3, 4, 4, 2, 2, 2, 2]]\n, Design a module that performs 32-bit addition using a 16-bit adder and a carry look-ahead adder. The module must be capable of taking two 32-bit inputs and providing a 32-bit output, which is the sum of the two input values.module top_module(\n    input [31:0] a,\n    input [31:0] b,\n    output [31:0] sum\n);"

In [29]:
new_conv_path = '/data/LPJ/ICML25/GraphCoder/graphgpt_dataset/gpt_dataset_construction/rtlcoder_gpt4_v2_with_module_head/import_for_graphgpt/conversations.json'
with open(new_conv_path, 'w') as f:
    json.dump(conv, f)